# Module 2: Path Following and Odometry

Here we will use the wheel's angular velocities and our robot commands to create a model for our For reference, they will be printed after running the code below. (You can copy and paste the output into the next part in Module 2), and follow a specified path.

This Module should follow Module 1: Basic Motion and Calibration.

### Modeling: Part 2 <a id="Modeling2"></a>

First, let's rerecord the measurements we measured in the first module. Enter your values in the code below. (If you copied the output at the end of Module 1, you can also paste that into this block).

In [ ]:
# TODO: Replace these values with what you found in Module 1
diameter = 0.06 # meters
length = 0.12 # meters
max_speed = 44.062 # rad/s

Next, we want to create a model for how our robot moves with this information. In order to accomplish this, we must first make two assumptions to simplify our equations.

1. The wheels do not skid (move sideways).
2. The wheels never slip.

With this, we can find the equations shown below regarding the relations between the angular velocity of the wheels and the velocities of the robot in the world frame.

$$\dot{x}=\frac{r}{2}\left(\omega_{R}+\omega_{L}\right)cos\left(\theta\right)
\\[2ex]
\dot{y}=\frac{r}{2}\left(\omega_{R}+\omega_{L}\right)sin\left(\theta\right)
\\[2ex]
\dot{\theta}=\frac{r}{L}\left(\omega_{R}-\omega_{L}\right)
$$

The variables ***x*** and ***y*** refers to the coordinates in the world frame that the robot is in, while theta (***$\theta$***) refers to the orientation of the robot, or which direction it is facing. The dots over the heads indicate that these are the speeds or velocities of these values.

We are almost able to calculate the position and orientation of our robot, but it still relies on one last piece of information to calculate. In order to find our place within a frame, we need our previous location and the time since the last position was calculated. These previous positions will be denoted as $x_{n-1}$, $y_{n-1}$, and $\theta_{n-1}$. The equations are similar to generic rate equations, and are as follows:

$$x_n=\dot{x}\left(t_n-t_{n-1}\right)+x_{n-1}
\\[2ex]
y_n=\dot{y}\left(t_n-t_{n-1}\right)+y_{n-1}
\\[2ex]
\theta_n=\dot{\theta}\left(t_n-t_{n-1}\right)+\theta_{n-1}
$$

Let's try out using these equations in a function now!

First, let's import our modules and initialize our robot.

In [ ]:
from jetbot import Robot
import time
import numpy as np

robot = Robot()
radius = diameter/2.0

curr_x = 0.0
curr_y = 0.0
curr_theta = 0
curr_time = None

Next, we will create the function to update our location whenever we call it.

<div class="alert alert-block alert-info">
Opportunity for Activity
</div>

In [ ]:
def update_pose(x, y, theta, t_prev):
    if t_prev is None:
        t_new = time.time()
        return (0.0, 0.0, 0, t_new)
    else:
        w_R = robot.right_motor.value*robot.max_speed
        w_L = robot.left_motor.value*robot.max_speed

        # TODO: Write the lines for these variables using the equations above
        # Note that you can use np.sin and np.cos for the trigonometric functions
        x_dot = radius/2*(w_R + w_L)*np.cos(theta)
        y_dot = radius/2*(w_R + w_L)*np.sin(theta)
        theta_dot = int(np.rad2deg(radius/length*(w_R - w_L)))

        t_new = time.time()
        dt = t_new - t_prev
        x_new = x_dot*dt + curr_x
        y_new = y_dot*dt + curr_y
        theta_new = ((theta_dot*dt + curr_theta) + 360) % 360
        return (x_new, y_new, theta_new, t_new)

### Path Following

Now that we have a way of determining position and orientation, we can attempt to create a path that our <b><span style="color:#154734">JetBot</span></b> follows. Our goal will be to drive a certain distance and stop once it reaches this. To accomplish this task, we must rely on our odometry to give feedback to our code as to where it is and stop when it reaches the desired position. This will create what is known as a closed loop system, where we have feedback from the robot helping us determine our motor commands.

To test the performance of your odometry, place one of the stop signs a certain distance away (anywhere from 2-5 meters) and see how accurate it is to stopping right in front of the sign you placed. The code below will test this performance, only the distance needs to be updated based on your environment.

<div class="alert alert-block alert-danger">
<b>WARNING!</b> The <b><span style="color:#154734">JetBot</span></b> will move when you run the following code.
</div>

In [ ]:
target_distance = 2.0 # meters

while(True):
    curr_x, curr_y, curr_theta, curr_time = update_pose(curr_x, curr_y, curr_theta, curr_time)
    robot.set_motors(.2, .2)
    dist_travelled = np.sqrt(np.square(curr_x) + np.square(curr_y))
    if dist_travelled >= target_distance:
        robot.stop()
        break
    time.sleep(0.02)

Hopefully, you should see that it stops right before the sign!

If you need to force it to stop, hit the square up top to interrupt the kernel, then running the following code to make your robot stop.

In [ ]:
robot.stop()